# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902030


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:01<00:28,  1.04it/s]

Rendering models:   9%|▉         | 3/32 [00:07<01:11,  2.48s/it]

Rendering models:  12%|█▎        | 4/32 [00:08<00:50,  1.80s/it]

Rendering models:  16%|█▌        | 5/32 [00:08<00:37,  1.40s/it]

...

Rendering models:  91%|█████████ | 29/32 [00:17<00:01,  2.73it/s]

Rendering models:  94%|█████████▍| 30/32 [00:17<00:00,  3.15it/s]

Rendering models:  97%|█████████▋| 31/32 [00:18<00:00,  1.98it/s]

Rendering models: 100%|██████████| 32/32 [00:19<00:00,  1.91it/s]

equidad1                              0.000590
ElisabethB                            0.001181
not-logged-in-aac1a549b28981db19c0    0.000642
CTidwell3                             0.000686
JessieDawn                            0.000791
not-logged-in-9f66093d0c2b8fe22169    0.000576
not-logged-in-0b0736c8991a7cc2017a    0.000578
Drew_Farnsworth                       0.024083
Thedunkmasta                          0.001407
mlang2017                             0.231551
not-logged-in-9454a205d149d7907cd0    0.022115
ericmalani                            0.001885
Stella07                              0.000561
jmartinez468                          0.003375
pangeli5                              0.000612
gabrielweiss                          0.001184
peakscience7                          0.000593
acapirala                             0.011037
Lavadude                              0.037101
helios1235                            0.000535
ShirleyL                              0.000581
Denny2507    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())